# Red ZorzNet CTC (Connectionist Temporal Classification)

## Puesta a punto

### Instalación de paquetes necesarios







In [0]:
!pip3 install python_speech_features

### Preparación del entorno para poder correr mis módulos importados

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

import sys
sys.path.append('drive/My Drive/Tesis')
sys.path.append('drive/My Drive/Tesis/repo')

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

## Entrenamiento

### Importación de librerías necesarias

In [0]:
import tensorflow as tf
from src.neural_network.ZorzNet.ZorzNetData import ZorzNetData
from src.neural_network.ZorzNet.ZorzNet import ZorzNet
from src.utils.Database import Database
from src.utils.ClassicLabel import ClassicLabel
from src.utils.ProjectData import ProjectData
import time
from src.Estimators.zorznet.model_fn import model_fn
from src.Estimators.zorznet.data_input_fn import data_input_fn

### Definición de los hiperparámetros de la red



In [0]:
project_data = ProjectData()

network_data = ZorzNetData()
network_data.model_path = 'drive/My Drive/Tesis/repo/' + project_data.ZORZNET_MODEL_PATH
network_data.checkpoint_path = 'drive/My Drive/Tesis/repo/' + project_data.ZORZNET_CHECKPOINT_PATH
network_data.tensorboard_path = 'drive/My Drive/Tesis/repo/' + project_data.ZORZNET_TENSORBOARD_PATH

network_data.num_classes = ClassicLabel.num_classes
network_data.num_features = 494

network_data.num_dense_layers_1 = 1
network_data.num_units_1 = [400]
network_data.dense_activations_1 = [tf.nn.relu] * network_data.num_dense_layers_1
network_data.batch_normalization_1 = True
network_data.keep_prob_1 = [0.6]
network_data.kernel_init_1 = [tf.truncated_normal_initializer(mean=0, stddev=0.1)] * network_data.num_dense_layers_1
network_data.bias_init_1 = [tf.zeros_initializer()] * network_data.num_dense_layers_1

network_data.is_bidirectional = True
# network_data.num_cell_units = [250]
# network_data.cell_activation = [tf.nn.tanh]
network_data.num_fw_cell_units = [256, 256]
network_data.num_bw_cell_units = [256, 256]
network_data.cell_fw_activation = [tf.nn.tanh] * 2
network_data.cell_bw_activation = [tf.nn.tanh] * 2
network_data.rnn_output_sizes = None

network_data.num_dense_layers_2 = 2
network_data.num_units_2 = [150, 100]
network_data.dense_activations_2 = [tf.nn.relu] * network_data.num_dense_layers_2
network_data.batch_normalization_2 = True
network_data.keep_prob_2 = [0.6, 0.6]
network_data.kernel_init_2 = [tf.truncated_normal_initializer(mean=0, stddev=0.1)] * network_data.num_dense_layers_2
network_data.bias_init_2 = [tf.zeros_initializer()] * network_data.num_dense_layers_2

network_data.dense_regularizer = 0.5
network_data.rnn_regularizer = 0.5
network_data.use_dropout = True

network_data.decoder_function = tf.nn.ctc_greedy_decoder

network_data.learning_rate = 0.001
network_data.adam_epsilon = 0.0001
network_data.optimizer = tf.train.AdamOptimizer(learning_rate=network_data.learning_rate, beta1=0.7, beta2=0.99)

### Configuración de entrenamiento

In [0]:
model_dir = 'drive/My Drive/Tesis/repo/out/zorznet/estimator/'

base_path = 'drive/My Drive/Tesis/repo/data/tfrecords/librispeech/classic/'

index_files = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
train_files = ['train_database_{}.tfrecords'.format(item) for item in index_files]
val_files = ['test_database_1.tfrecords', 'test_database_2.tfrecords']
test_files = ['test_database_2.tfrecords']

train_files = list(map(lambda x: base_path + x, train_files))
validate_files = list(map(lambda x: base_path + x, val_files))
test_files = list(map(lambda x: base_path + x, test_files))

train_batch_size = 100
train_epochs = 1000

validate_batch_size = 100
               
restore_run = True

config = tf.estimator.RunConfig(
    model_dir=model_dir,
    save_checkpoints_steps=100,
    save_summary_steps=100,
    log_step_count_steps=100)


model = tf.estimator.Estimator(
    model_fn=model_fn,
    params=network_data.as_dict(),
    config=config
)

tf.logging.set_verbosity(tf.logging.INFO)

if not restore_run:
    shutil.rmtree(model_dir)

### Entrenamiento

In [0]:
model.train(
    input_fn=lambda: data_input_fn(
        filenames=train_files,
        batch_size=train_batch_size,
        parse_fn=Database.tfrecord_parse_dense_fn,
        shuffle_buffer=10,
        num_features=network_data.num_features,
        num_epochs=1)
)


### Validación


In [0]:
model.evaluate(
    input_fn=lambda: data_input_fn(
        filenames=validate_files,
        batch_size=validate_batch_size,
        parse_fn=Database.tfrecord_parse_dense_fn,
        shuffle_buffer=1,
        num_features=network_data.num_features)
)

### Testeo

In [0]:
num_tests = 10

predictions = model.predict(
    input_fn=lambda: data_input_fn(
        filenames=test_files,
        batch_size=1,
        parse_fn=Database.tfrecord_parse_dense_fn,
        shuffle_buffer=1,
        num_features=network_data.num_features)
)

for item in predictions:
  print("Predicted: " + ClassicLabel.from_index(item))
  num_tests -= 1
  if num_tests == 0:
    break